In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

from __future__ import print_function, division 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv) 
import torch
import torch.nn as nn
from torch.optim import lr_scheduler
import torchvision
from torchvision import models, datasets, transforms
import matplotlib.pyplot as plt
import time 
import shutil
import copy 
from IPython.display import Image


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Load Data

In [ ]:
metadata = pd.read_csv("../input/mayo-clinic-strip-ai-jpg-dataset/data/train.csv")
img_id = metadata.iloc[:, 0]
labels = metadata.iloc[:, 4]
img_to_lbl = pd.concat([img_id, labels], axis = 1)
img_to_lbl.set_index('image_id')
img_to_lbl = img_to_lbl.to_dict()
img_to_lbl


In [57]:
from glob import glob
from collections import defaultdict
Image.MAX_IMAGE_PIXELS = None 
train_images = glob("/kaggle/input/mayo-clinic-strip-ai/train/*")


img_features = defaultdict(list)

for img_path in train_images:
    img = Image.open(img_path)
    img_features['image_id'].append(img_path[-12:-4])
    img_features['width'].append(img.size[0])
    img_features['height'].append(img.size[1])
    img_features['img_path'].append(img_path)
    
img_data = pd.DataFrame(img_features)
img_data.head()

""


In [ ]:
os.getcwd()

In [ ]:
pip install split-folders

In [ ]:
import splitfolders as spf

In [ ]:
spf.fixed("../input/mayo-clinic-strip-ai-jpg-dataset/data/train",
          output = "output", 
          seed = 1337, 
          fixed = (630, 100), 
          oversample = True, 
          group_prefix = None, 
          move = False)
                   

In [ ]:
data_dir = '../input/mayo-clinic-strip-ai-jpg-dataset'
image_data = {x: datasets.ImageFolder(data_dir) for x in ['train']}
dataloader = {x: torch.utils.data.DataLoader(image_data[x], batch_size=4,
                                             shuffle=True, num_workers=4)
              for x in ['train']}
class_names = {'CE': 0, 'LAA': 1}

In [ ]:
def imshow(): 
    